# Download requirements

In [2]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Individual sampling funtions

In [30]:
# Generate response using beam search
def generate_response(model, tokenizer, prompt, max_length, num_beams, num_return_sequences):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    # Perform beam search using the generate method
    outputs = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,  # Number of sequences to return
        output_scores=True,      # Return the scores of generated sequences
        return_dict_in_generate=True,  # Return a dictionary
    )

    # Decode the output sequences
    responses = [tokenizer.decode(seq, skip_special_tokens=True) for seq in outputs.sequences]
    return responses

In [10]:
import torch
import torch.nn.functional as F

def keyword_sample_sequence(model, tokenizer, input_text, max_length=50, temperature=1.0):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    generated_tokens = []
    token_dict = {}

    # All operators and keywords
    logical_operators = {'and', 'or', 'not'}
    keywords = {'for', 'is', 'in', 'def', 'while', 'return', 'with'}
    arithmetic_operators = {'+', '-', '*', '/', '%'}
    relational_operators = {'==', '!=', '<', '>', '<=', '>='}

    for _ in range(max_length):
        # Get logits from the model for the last token in input_ids
        logits = model(input_ids)[0][:, -1, :] / temperature
        probabilities = F.softmax(logits, dim=-1).squeeze()

        # Get top probability and token
        top_prob, top_index = torch.max(probabilities, dim=-1)
        top_token = tokenizer.decode([top_index.item()]).strip()

        # Determine the threshold based on the type of token
        if top_token in logical_operators:
            threshold = 1
            color_code = "\033[92m"  # Green
        elif top_token in arithmetic_operators:
            threshold = 1
            color_code = "\033[93m"  # Yellow
        elif top_token in keywords:
            threshold = 1
            color_code = "\033[94m"  # Blue
        elif top_token in relational_operators:
            threshold = 1
            color_code = "\033[91m"  # Red
        else:
            threshold = 0.6  # Default threshold for other tokens
            color_code = "\033[96m"  # Cyan

        # Sort probabilities and their corresponding indices
        sorted_probs, sorted_indices = torch.sort(probabilities, descending=True)
        print("Top 3 probabilities:", sorted_probs[:3])
        print("Top 3 indices:", sorted_indices[:3])
        print("Top 3 tokens:", tokenizer.decode(sorted_indices[:3]))

        for i in range(3):
            print(f"{i + 1}: {tokenizer.decode(sorted_indices[i])}")

        if top_prob.item() < threshold and sorted_probs[0] - sorted_probs[1] > 0.1:
            # Remove the highest probability token
            color = True
            sorted_probs = sorted_probs[1:]  # Remove the highest probability
            sorted_indices = sorted_indices[1:]  # Remove the highest probability
            sorted_probs /= torch.sum(sorted_probs)  # Re-normalize probabilities
            print(f"inside the condition top prob is less than {threshold}")

            # Sample from remaining tokens
            next_token_idx = torch.multinomial(sorted_probs, num_samples=1).item()
            next_token = sorted_indices[next_token_idx].item()
        else:
            color = False
            # Sample normally from original probabilities
            next_token = torch.multinomial(probabilities, num_samples=1).item()

        # Store the individual token to a dict
        token_dict[next_token] = tokenizer.decode([next_token])

        # Append next token to input_ids
        input_ids = torch.cat([input_ids, torch.tensor([[next_token]]).to(device)], dim=-1)
        generated_tokens.append((next_token, color, color_code if color else ""))  # Store the token, color flag, and color code

        # Decode generated tokens so far
        generated_text = tokenizer.decode(input_ids.squeeze().cpu())

        # Display the generated text
        for token_id, is_colored, color_code in generated_tokens:
            token_text = tokenizer.decode([token_id])
            if is_colored:
                print(f"{color_code}{token_text}\033[0m", end='')
            else:
                print(token_text, end='')
        print("\n-------------------------------------------------------")

        # Check if generated text contains end-of-sequence token
        if tokenizer.eos_token_id is not None and next_token == tokenizer.eos_token_id:
            break

    # Return final generated text
    final_generated_text = ""
    for token_id, is_colored, color_code in generated_tokens:
        token_text = tokenizer.decode([token_id])
        if is_colored:
            final_generated_text += f"{color_code}{token_text}\033[0m"
        else:
            final_generated_text += token_text

    return final_generated_text, token_dict


In [11]:
input_text = """Define fibonacci series in python Dont give me any comments in code"""
generated_text, all_token = keyword_sample_sequence(model, tokenizer, input_text, max_length=500, temperature=0.5)
print("Generated text:\n", generated_text)

Top 3 probabilities: tensor([0.4412, 0.3894, 0.0985], device='cuda:0', grad_fn=<SliceBackward0>)
Top 3 indices: tensor([ 13, 185, 188], device='cuda:0')
Top 3 tokens: .

1: .
2: 

3: 
.
-------------------------------------------------------
Top 3 probabilities: tensor([0.9887, 0.0067, 0.0015], device='cuda:0', grad_fn=<SliceBackward0>)
Top 3 indices: tensor([185, 207, 303], device='cuda:0')
Top 3 tokens: 
  I
1: 

2:  
3:  I
.

-------------------------------------------------------
Top 3 probabilities: tensor([0.7262, 0.1620, 0.0596], device='cuda:0', grad_fn=<SliceBackward0>)
Top 3 indices: tensor([185, 315, 184], device='cuda:0')
Top 3 tokens: 
   	
1: 

2:    
3: 	
.


-------------------------------------------------------
Top 3 probabilities: tensor([0.8232, 0.1114, 0.0151], device='cuda:0', grad_fn=<SliceBackward0>)
Top 3 indices: tensor([185, 315, 184], device='cuda:0')
Top 3 tokens: 
   	
1: 

2:    
3: 	
.

   
-------------------------------------------------------
Top 3 pr

In [ ]:
# Example usage
prompt = "Write a python code for checking if a number is even or odd.Dont give any english explanation"
max_length = 500
num_beams = 10
num_return_sequences = 10  # Retrieve the top 2 sequences

responses = generate_response(model, tokenizer, prompt, max_length, num_beams, num_return_sequences)
print("Best response:", responses[0])
print("*"*100)
print("Second best response:", responses[1])
print("*"*100)
print("Third response", responses[2])
print("*"*100)
print("Forth response", responses[9])


## Tesing sampling

# Class based Code

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn.functional as F

class DeepSeekQgen:
    
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def keyword_sample_sequence(self, model, tokenizer,input_text, max_length=50, temperature=1.0):
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(self.device)
        generated_tokens = []
        token_dict = {}

        # Define operators and keywords
        logical_operators = {'and', 'or', 'not'}
        keywords = {'for', 'is', 'in', 'def', 'while', 'return', 'with'}
        arithmetic_operators = {'+', '-', '*', '/', '%'}
        relational_operators = {'==', '!=', '<', '>', '<=', '>='}

        for _ in range(max_length):
            # Get logits from the model for the last token in input_ids
            logits = model(input_ids).logits[:, -1, :] / temperature
            probabilities = F.softmax(logits, dim=-1).squeeze()

            # Get top probability and token
            top_prob, top_index = torch.max(probabilities, dim=-1)
            top_token = tokenizer.decode([top_index.item()]).strip()

            # Determine the threshold based on the type of token
            if top_token in logical_operators:
                threshold = 1
                color_code = "\033[92m"  # Green
            elif top_token in arithmetic_operators:
                threshold = 1
                color_code = "\033[93m"  # Yellow
            elif top_token in keywords:
                threshold = 1
                color_code = "\033[94m"  # Blue
            elif top_token in relational_operators:
                threshold = 1
                color_code = "\033[91m"  # Red
            else:
                threshold = 0.8  # Default threshold for other tokens
                color_code = "\033[96m"  # Cyan

            # Sort probabilities and their corresponding indices
            sorted_probs, sorted_indices = torch.sort(probabilities, descending=True)
            print("Top 3 probabilities:", sorted_probs[:3])
            print("Top 3 indices:", sorted_indices[:3])
            print("Top 3 tokens:", tokenizer.decode(sorted_indices[:3]))

            for i in range(3):
                print(f"{i + 1}: {tokenizer.decode(sorted_indices[i])}")

            if top_prob.item() < threshold and sorted_probs[0] - sorted_probs[1] > 0.1:
                # Remove the highest probability token
                color = True
                sorted_probs = sorted_probs[1:]  # Remove the highest probability
                sorted_indices = sorted_indices[1:]  # Remove the highest probability
                sorted_probs /= torch.sum(sorted_probs)  # Re-normalize probabilities
                print(f"Inside the condition top prob is less than {threshold}")

                # Sample from remaining tokens
                next_token_idx = torch.multinomial(sorted_probs, num_samples=1).item()
                next_token = sorted_indices[next_token_idx].item()
            else:
                color = False
                # Sample normally from original probabilities
                next_token = torch.multinomial(probabilities, num_samples=1).item()

            # Store the individual token to a dict
            token_dict[next_token] = tokenizer.decode([next_token])

            # Append next token to input_ids
            input_ids = torch.cat([input_ids, torch.tensor([[next_token]]).to(self.device)], dim=-1)
            generated_tokens.append((next_token, color, color_code if color else ""))  # Store the token, color flag, and color code

            # Decode generated tokens so far
            generated_text = tokenizer.decode(input_ids.squeeze().cpu())

            # Display the generated text
            for token_id, is_colored, color_code in generated_tokens:
                token_text = tokenizer.decode([token_id])
                if is_colored:
                    print(f"{color_code}{token_text}\033[0m", end='')
                else:
                    print(token_text, end='')
            print("\n-------------------------------------------------------")

            # Check if generated text contains end-of-sequence token
            if tokenizer.eos_token_id is not None and next_token == tokenizer.eos_token_id:
                break

        # Return final generated text
        final_generated_text = ""
        for token_id, is_colored, color_code in generated_tokens:
            token_text = tokenizer.decode([token_id])
            if is_colored:
                final_generated_text += f"{color_code}{token_text}\033[0m"
            else:
                final_generated_text += token_text

        return final_generated_text, token_dict

    def gpu_sample_sequence(self, input_text, model, tokenizer, max_length=50, temperature=1.0):
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(self.device)
        generated_tokens = []
        token_dict = {}

        for _ in range(max_length):
            # Get logits from the model for the last token in input_ids
            logits = model(input_ids).logits[:, -1, :] / temperature
            probabilities = F.softmax(logits, dim=-1).squeeze()

            top_prob = torch.max(probabilities).item()

            # Sort probabilities and their corresponding indices
            sorted_probs, sorted_indices = torch.sort(probabilities, descending=True)
            print("Top 3 probabilities:", sorted_probs[:3])
            print("Top 3 indices:", sorted_indices[:3])
            print("Top 3 tokens:", tokenizer.decode(sorted_indices[:3]))

            for i in range(3):
                print(f"{i + 1}: {tokenizer.decode(sorted_indices[i])}")

            if top_prob < 0.95 and sorted_probs[0] - sorted_probs[1] > 0.1:
                # Remove the highest probability token
                color = True
                sorted_probs = sorted_probs[1:]  # Remove the highest probability
                sorted_indices = sorted_indices[1:]  # Remove the highest probability
                sorted_probs /= torch.sum(sorted_probs)  # Re-normalize probabilities
                print("Inside the condition top prob is less than 0.95")

                # Sample from remaining tokens
                next_token_idx = torch.multinomial(sorted_probs, num_samples=1).item()
                next_token = sorted_indices[next_token_idx].item()
            else:
                color = False
                # Sample normally from original probabilities
                next_token = torch.multinomial(probabilities, num_samples=1).item()

            # Store the individual token to a dict
            token_dict[next_token] = tokenizer.decode([next_token])

            # Append next token to input_ids
            input_ids = torch.cat([input_ids, torch.tensor([[next_token]]).to(self.device)], dim=-1)
            generated_tokens.append((next_token, color))  # Store the token and color flag

            # Decode generated tokens so far
            generated_text = tokenizer.decode(input_ids.squeeze().cpu())

            # Display the generated text
            for token_id, is_colored in generated_tokens:
                token_text = tokenizer.decode([token_id])
                if is_colored:
                    print(f"\033[91m{token_text}\033[0m", end='')
                else:
                    print(token_text, end='')
            print("\n-------------------------------------------------------")

            # Check if generated text contains end-of-sequence token
            if self.tokenizer.eos_token_id is not None and next_token == self.tokenizer.eos_token_id:
                break

        # Return final generated text
        final_generated_text = ""
        for token_id, is_colored in generated_tokens:
            token_text = tokenizer.decode([token_id])
            if is_colored:
                final_generated_text += f"\033[91m{token_text}\033[0m"
            else:
                final_generated_text += token_text

        return final_generated_text, token_dict


In [26]:
def download_model(model_name="deepseek-ai/deepseek-coder-1.3b-instruct", use_bfloat16=True):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if use_bfloat16:
        model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.bfloat16)
    else:
        model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
    model.to(device)
    print("Model and tokenizer initialized.")
    return model, tokenizer

### Create Object and download model

In [29]:
# download model
model,tokenizer = download_model()

qgen = DeepSeekQgen()
input_text = "write a python prgram for fibbonci series give me the code without any comment and explanation "

print("Keyword Sample Sequence:")
generated_text, token_dict = qgen.keyword_sample_sequence(model=model, tokenizer=tokenizer, input_text=input_text, max_length=500, temperature=0.5)
print(generated_text)

# print("\nGPU Sample Sequence:")
# generated_text, token_dict = qgen.gpu_sample_sequence(model=model, tokenizer=tokenizer, input_text=input_text, max_length=500, temperature=0.5)
# print(generated_text)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model and tokenizer initialized.
Keyword Sample Sequence:
Top 3 probabilities: tensor([9.9750e-01, 2.4726e-03, 1.6660e-05], device='cuda:0',
       grad_fn=<SliceBackward0>)
Top 3 indices: tensor([185, 188, 327], device='cuda:0')
Top 3 tokens: 
 for
1: 

2: 
3:  for


-------------------------------------------------------
Top 3 probabilities: tensor([0.5754, 0.2117, 0.0779], device='cuda:0', grad_fn=<SliceBackward0>)
Top 3 indices: tensor([185, 315, 184], device='cuda:0')
Top 3 tokens: 
   	
1: 

2:    
3: 	
Inside the condition top prob is less than 0.8

   
-------------------------------------------------------
Top 3 probabilities: tensor([0.8830, 0.0725, 0.0342], device='cuda:0', grad_fn=<SliceBackward0>)
Top 3 indices: tensor([ 972,  185, 1494], device='cuda:0')
Top 3 tokens:  def
 #
1:  def
2: 

3:  #
Inside the condition top prob is less than 1

    #
-------------------------------------------------------
Top 3 probabilities: tensor([0.1252, 0.0975, 0.0670], device='cuda:0', g

## Testing

In [12]:
def fibo(n):
    fib = [0,1]
    while len(fib)<=n:
       fib.append(fib[-1] + fib[-2])
    print(fib)
                 
fibo(5)

[0, 1, 1, 2, 3, 5]


# New Approach

In [14]:
messages=[
    { 'role': 'user', 'content': """Write a Python class in which there should be different function for different task 
so first you have to calculate the nth fibbonci number then finding the largest prime number 
and then convert that largest prime into binary from that then set the 2nd bit and give me the output from that 

So first give me the pseudocode then whole code which i can directly execute
give me these without any english explanation and comments"""}
]

inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, temperature = 0.5, max_new_tokens=1024, do_sample=True, top_k=5, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id,output_logits=True,output_scores=True,return_dict_in_generate=True)

print(tokenizer.decode(outputs.sequences[0][len(inputs[0]):]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Here is the pseudocode for the problem:

1. Define a class FibonacciPrime.
2. In the constructor of the class, accept an argument n.
3. Define a method to calculate the nth Fibonacci number.
4. Define a method to find the largest prime number less than or equal to the calculated Fibonacci number.
5. Define a method to convert the largest prime number into binary.
6. Define a method to set the 2nd bit of the binary representation of the largest prime number.
7. Define a method to return the output from the set 2nd bit.

Pseudocode:

```
class FibonacciPrime:
    def __init__(self, n):
        self.n = n

    def fibonacci(self, n):
        if n <= 0:
            return 0
        elif n == 1:
            return 1
        else:
            return self.fibonacci(n-1) + self.fibonacci(n-2)

    def largest_prime(self):
        num = self.fibonacci(self.n)
        while num > 1:
            if all(num % i != 0 for i in range(2, int(num**0.5) + 1)):
                return num
            num 

In [6]:
messages=[
    { 'role': 'user', 'content': """Write a Python class in which there should be different function for different task 
Pseudocode:

1. Define a function to find the largest prime number from the Fibonacci number.
2. Define a function to calculate the nth Fibonacci number.
3. Define a function to set the 2nd bit of the binary number.
4. Define a function to return the output from the 2nd bit of the binary number.
5. Define a function to convert the Fibonacci number to binary.
give me these without any english explanation and comments"""}
]

inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, temperature = 0.5, max_new_tokens=512, do_sample=True, top_k=5, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id,output_logits=True,output_scores=True,return_dict_in_generate=True)

print(tokenizer.decode(outputs.sequences[0][len(inputs[0]):]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Sure, here's a Python class with the requested functions:

```python
class Fibonacci:
    def __init__(self):
        self.fib_cache = {}

    def largest_prime_fibonacci(self, n):
        # Find the largest prime number from the Fibonacci number
        def is_prime(num):
            if num < 2:
                return False
            for i in range(2, int(num**0.5) + 1):
                if num % i == 0:
                    return False
            return True

        fib_numbers = [self.fib(n) for n in range(n+1)]
        largest_prime = max(filter(is_prime, fib_numbers))
        return largest_prime

    def fib(self, n):
        # Calculate the nth Fibonacci number
        if n in self.fib_cache:
            return self.fib_cache[n]
        if n <= 2:
            return 1
        self.fib_cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.fib_cache[n]

    def set_bit(self, num, pos):
        # Set the 2nd bit of the binary number
        return num | (1 << pos)

    de

In [12]:
class Fibonacci:
    def __init__(self):
        self.fib_cache = {}

    def largest_prime_fibonacci(self, n):
        # Find the largest prime number from the Fibonacci number
        def is_prime(num):
            if num < 2:
                return False
            for i in range(2, int(num**0.5) + 1):
                if num % i == 0:
                    return False
            return True

        fib_numbers = [self.fib(n) for n in range(n+1)]
        largest_prime = max(filter(is_prime, fib_numbers))
        return largest_prime

    def fib(self, n):
        if n in self.fib_cache:
            return self.fib_cache[n]
        if n <= 2:
            return 1
        self.fib_cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.fib_cache[n]

    def set_bit(self, num, pos):
        # Set the 2nd bit of the binary number
        return num | (1 << pos)

    def get_bit(self, num, pos):
        # Return the output from the 2nd bit of the binary number
        return (num >> pos) & 1

    def fib_to_binary(self, n):
        # Convert the Fibonacci number to binary
        binary = bin(self.fib(n))[2:]
        return binary

obj = Fibonacci()
print(obj.largest_prime_fibonacci(4))
print(obj.set_bit(10,2))

3
14


In [24]:
class FibonacciPrime:
    def __init__(self, n):
        self.n = n

    def fibonacci(self, n):
        if n <= 0:
            return 0
        elif n == 1:
            return 1
        else:
            return self.fibonacci(n-1) + self.fibonacci(n-2)

    def largest_prime(self):
        num = self.fibonacci(self.n)
        while num > 1:
            if all(num % i != 0 for i in range(2, int(num**0.5) + 1)):
                return num
            num -= 1
        return num

    def prime_to_binary(self):
        prime = self.largest_prime()
        binary = bin(prime)[2:]
        return binary

    def set_second_bit(self):
        binary = self.prime_to_binary()
        binary = binary[:7] + '1' + binary[7:]
        return binary

    def output(self):
        return self.set_second_bit()

# Test the class
fibonacci_prime = FibonacciPrime(10)
print(fibonacci_prime.fibonacci(10))
print(fibonacci_prime.largest_prime())
# print(fibonacci_prime.output())

55
53
